#### ROUTING GAME LIBRARY

In [1]:
include("routing_games.jl")

plot_labeled (generic function with 1 method)

#### FORWARD SOLVER: game(p,E,s, b, C) ==> (x, v)

In [2]:
include("routing_solvers/routing_solver_entropy_jump.jl")
include("routing_solvers/routing_solver_entropy_nlsolve.jl")
include("routing_solvers/routing_solver_exact_jump.jl")

solve_routing (generic function with 1 method)

##### test forward solvers

In [3]:
pa = grid_graph3_4_players_reduced()
b = 0.1*ones(pa.p*pa.m)
C = zeros(pa.p*pa.m, pa.p*pa.m)
x_init = rand(pa.p*pa.m)
v_init = zeros(pa.p*(pa.n-1))

x, v = solve_entropy_routing_jump(pa, b, C, 0.01, x_init, v_init)
x, v = solve_entropy_routing_nlsolve(pa, b, C, 0.01, x_init, v_init)
x, v = solve_routing(pa, b, C, x_init, v_init)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

			--- entropy routing solver (JuMP - Ipopt) ---
			 termination_status(model) = LOCALLY_SOLVED
			 objective_value(model) = 0.0003713429296991849
			 solve_time(model) = 1.0620551109313965
			--- entropy routing solver (NLsolve) ---
			--- exact routing solver ---
			 termination_status(model) = LOCALLY_SOLVED
			 objective_value(model) = -1.7512476602479265e-7
			 solve_time(model) = 0.018352985382080078


(x = [-9.81626910665262e-9, -9.939904065971528e-9, -9.938923058324453e-9, -9.816049716983657e-9, -9.909582564764424e-9, -9.939390610500222e-9, -9.817105532896943e-9, -9.817250114299693e-9, 0.9999999898009964, -9.817187266941606e-9  …  -9.81353228921262e-9, -9.909147681253437e-9, -9.93583693895067e-9, -9.817621691449843e-9, -9.935464402870628e-9, 0.9999999897987724, -9.811728972422793e-9, -9.81991331477345e-9, -9.812719338037414e-9, -9.943030915686704e-9], v = [0.1500000149863635, 0.10000000999090924, 0.05000000499545462, 0.20000001998181818, 0.10000000999090909, 0.15000001498636356, 0.10000000999090895, 0.050000004995454275, 0.050000004995454546, 0.10000000999090909  …  0.05000000499545455, 0.05000000499545482, 0.05000000499545482, 0.050000004995454546, 0.10000000999090923, 0.10000000999090909, 0.10000000999090909, 0.1500000149863637, 0.20000001998181818, 0.15000001498636362])

#### BACKWARD SOLVER: (p,E,s, x_hat) ==> (b, C)

In [4]:
include("approx_proj_gradient.jl")

approx_proj_grad

##### test backward solver

In [5]:
using ArgParse
using JLD2
using Plots, GraphPlot, ColorSchemes

# parse cmd args
function parse_commandline()
    s = ArgParseSettings()

    @add_arg_table s begin
        "--rho"
            help = "rho"
            arg_type = Float64
            default = 0.5
        "--lambda"
            help = "entropy reg term"
            arg_type = Float64
            default = 0.01
        "--alpha"
            help = "GD step size"
            arg_type = Float64
            default = 0.005
        "--epsilon"
            help = "convergence threshold"
            arg_type = Float64
            default = 1e-3
        "--max_iter"
            help = "maximum iter allowed"
            arg_type = Int64
            default = 10
        "--plot"
            help = "plot exp result or not"
            arg_type = Bool
            default = true
        "--forward_solver"
            help = "nlsolve or ipopt"
            arg_type = String
            default = "nlsolve"
    end

    return parse_args(s)
end
args = parse_commandline()

Dict{String, Any} with 7 entries:
  "max_iter"       => 10
  "alpha"          => 0.005
  "epsilon"        => 0.001
  "plot"           => true
  "lambda"         => 0.01
  "forward_solver" => "nlsolve"
  "rho"            => 0.5

In [6]:
pa = grid_graph3_4_players_reduced()

x = zeros(pa.p*pa.m)
v = zeros(pa.p*(pa.n-1))
b = 0.1*ones(pa.p*pa.m)
C = zeros(pa.p*pa.m, pa.p*pa.m)

# assign parameters
λ = 0.01
α = 0.005
ϵ = 1e-3
ρ = 0.5
max_iter = 10

println("----------- $(pa.game_name)_λ=($λ)_α=($α)_ϵ=($ϵ)_ρ=($ρ) -----------")
x, v, b, C, lambda_vals, ψ_vals_exact = approx_proj_grad(pa, λ, α, ϵ, ρ, max_iter, x, v, b, C)

----------- grid_graph3_4_players_reduced_λ=(0.01)_α=(0.005)_ϵ=(0.001)_ρ=(0.5) -----------
		### ITER 1 ###

			*** Evaluate (b, C) => (x_exact, v_exact) => ψ(x_exact) for iter 1 ***
			--- exact routing solver ---
			 termination_status(model) = LOCALLY_SOLVED
			 objective_value(model) = -1.3067156341955138e-7
			 solve_time(model) = 0.0140380859375
			 ψ_x_exact = 6.000000400823994

			*** Compute (x, v, λ = 0.01) => (b, C) in iter 1 ***
			update b, C...
			--- entropy routing solver (NLsolve) ---
			compute D, J...
			compute ∇ψ_x...
			compute ∇̂ψ_b, ∇ψ_C...
			compute b_plus, C_plus...

		### ITER 2 ###

			*** Evaluate (b, C) => (x_exact, v_exact) => ψ(x_exact) for iter 2 ***
			--- exact routing solver ---
			 termination_status(model) = LOCALLY_SOLVED
			 objective_value(model) = -1.7512677912262241e-7
			 solve_time(model) = 0.027026891708374023
			 ψ_x_exact = 12.00000007547728

			*** Compute (x, v, λ = 0.01) => (b, C) in iter 2 ***
			update b, C...
			--- entropy routing

(x = [0.9757900689543461, 2.2387823345541093e-9, 2.23878227537221e-9, 0.9757900688121318, 1.6696773499201868e-5, 2.238780192162656e-9, 0.9757900688121318, 0.9757900689543461, 0.021505056247799574, 0.0027048792757191423  …  0.0027048789952502554, 1.6696756248184654e-5, 2.238782518950796e-9, 0.9757900719587487, 2.238782473006169e-9, 0.02150505340279034, 0.9757900719587488, 0.0027048791163261153, 0.0027048791163260697, 2.238782564190832e-9], v = [0.267520036624824, 0.17834669742559064, 0.08917334875521864, 0.35669337576738064, 0.1783466658333573, 0.2675200371084813, 0.17834669834856495, 0.08917334924957132, 0.08917334941593817, 0.17834669868129863  …  0.08917334914808404, 0.08917334865373096, 0.08917334813615128, 0.08917334863050427, 0.1783466961874569, 0.17834666459522353, 0.17834669711043075, 0.26752003476762276, 0.3566933732911124, 0.26752003525128026], b = [0.04500512603486366, 0.09999999970775149, 0.09999999970775221, 0.04500510231777385, 0.1, 0.09999999970775232, 0.04500510209555478

#### Experiment

In [7]:
function homotopy_exp_parameter_choice(pa, α, ϵ, ρ, max_iter, λ_list)

    # output placeholder
    x = zeros(pa.p*pa.m)
    v = zeros(pa.p*(pa.n-1))
    b = 0.1*ones(pa.p*pa.m)
    C = zeros(pa.p*pa.m, pa.p*pa.m)
    lambda_vals_list = Vector{Float64}[]
    ψ_vals_exact_list = Float64[]
    ψ_x_exact = Inf

    # run experiment
    println("----------- $(pa.game_name)_ρ=($ρ)_λ_list=($λ_list)_α=($α)_ϵ=($ϵ) -----------\n")
    for λ in λ_list
        if ψ_x_exact <= ϵ
            println("\t----- approx_proj_grad converged -----")
            break
        else
            println("\t----- approx_proj_grad with {λ=$λ} -----\n")
            x, v, b, C, lambda_vals, ψ_vals_exact = approx_proj_grad(pa, λ, α, ϵ, ρ, max_iter, x, v, b, C)
            push!(lambda_vals_list, lambda_vals)
            append!(ψ_vals_exact_list, ψ_vals_exact)
            ψ_x_exact = ψ_vals_exact_list[end]
        end
    end
    println("--------------------------------\n")

     # create an individual folder under homotopy_results/
    dir = "homotopy_results/$(pa.game_name)"
    mkpath(dir) # mkdir if not exists

    # save data to folder
    println("saving result...")
    @save "$dir/$(pa.game_name)_λ_list=($λ_list)_homotopy.jld2" ρ λ_list α ϵ max_iter x v b C lambda_vals_list ψ_vals_exact_list
    println("saved result to '$dir/$(pa.game_name)_λ_list=($λ_list)_homotopy.jld2'")
    println("--------------------------------\n")

    (;x = x,
      v = v,
      b = b,
      C = C,
      lambda_vals_list = lambda_vals_list,
      ψ_vals_exact_list = ψ_vals_exact_list,
      dir = dir)
end

homotopy_exp_parameter_choice (generic function with 1 method)

In [8]:
""" RUNNING EXP: PARAMETER CHOICE """
# assign parameters
α = args["alpha"]
ϵ = args["epsilon"]
ρ = args["rho"]
max_iter = args["max_iter"]
λ_list = [1.0, 0.5, 0.1, 0.02]

# calling method
x, v, b, C, lambda_vals_list, ψ_vals_exact_list, dir = homotopy_exp_parameter_choice(grid_graph3_4_players_reduced(), α, ϵ, ρ, max_iter, λ_list)

----------- grid_graph3_4_players_reduced_ρ=(0.5)_λ_list=([1.0, 0.5, 0.1, 0.02])_α=(0.005)_ϵ=(0.001) -----------

	----- approx_proj_grad with {λ=1.0} -----

		### ITER 1 ###

			*** Evaluate (b, C) => (x_exact, v_exact) => ψ(x_exact) for iter 1 ***
			--- exact routing solver ---
			 termination_status(model) = LOCALLY_SOLVED
			 objective_value(model) = -1.3067156341955138e-7
			 solve_time(model) = 0.014579057693481445
			 ψ_x_exact = 6.000000400823994

			*** Compute (x, v, λ = 1.0) => (b, C) in iter 1 ***
			update b, C...
			--- entropy routing solver (NLsolve) ---
			compute D, J...
			compute ∇ψ_x...
			compute ∇̂ψ_b, ∇ψ_C...
			compute b_plus, C_plus...

		### ITER 2 ###

			*** Evaluate (b, C) => (x_exact, v_exact) => ψ(x_exact) for iter 2 ***
			--- exact routing solver ---
			 termination_status(model) = LOCALLY_SOLVED
			 objective_value(model) = -1.7512677912262241e-7
			 solve_time(model) = 0.0185239315032959
			 ψ_x_exact = 12.00000007547728

			*** Compute (x, v, λ = 1

(x = [0.595642463124329, 0.17577323377291965, 0.1757732404743918, 0.5956424743358766, 0.2609973868726792, 0.17577325230457472, 0.5956424677085738, 0.5956424564228568, 0.4991146783470488, 0.37264606898838587  …  0.37264606840256925, 0.26099739297190394, 0.1757732408840804, 0.5956424631243291, 0.17577323377291967, 0.49911467834704887, 0.595642456422857, 0.3726460689883859, 0.3726460729426109, 0.17577323692985541], v = [0.6694943775052549, 0.44632957881405616, 0.2231647929782732, 0.8926591834187023, 0.44632957859772004, 0.6694943821607344, 0.4463295838740843, 0.22316479340497128, 0.2231647934049712, 0.44632958387408433  …  0.2231647934049712, 0.22316479297827324, 0.22316479297827327, 0.22316479340497125, 0.44632957881405616, 0.44632957859772004, 0.44632958387408445, 0.669494377505255, 0.8926591834187023, 0.6694943821607344], b = [0.08159130934521988, 0.1, 0.1, 0.0815913071245839, 0.1, 0.1, 0.08159130811997951, 0.0815913103514009, 0.1, 0.1  …  0.1, 0.1, 0.1, 0.0815913093452199, 0.1, 0.1, 0

In [9]:
using IJulia

pa = grid_graph3_4_players_reduced()

@load "$dir/$(pa.game_name)_λ_list=($λ_list)_homotopy.jld2" ρ λ_list α ϵ max_iter x v b C lambda_vals_list ψ_vals_exact_list

In [ ]:
# plot single axis (log-scaled)
println("plotting homotopy...")
p1 = plot(title="ρ=($ρ), α=($α) \n λ_list=($λ_list)")
lambda_vals_list_flattened = collect(Iterators.flatten(lambda_vals_list))
lambda_colors = palette([:green, :blue], length(lambda_vals_list))
for i in eachindex(lambda_vals_list)
    lambda_vals = lambda_vals_list[i]
    plot!(findall(x->x==lambda_vals[1], lambda_vals_list_flattened), lambda_vals, yscale=:log10, color = lambda_colors[i], marker=:circle, label="λ=$(lambda_vals[1])", leg=:bottomleft)
end 
plot!(ψ_vals_exact_list, yscale=:log10, c=:red, linestyle=:dash, label="ψ(x)_exact", leg=:bottomleft)
hline!([ϵ], label="ϵ=$ϵ", c=:grey)
savefig("$dir/$(pa.game_name)_λ_list=($λ_list)_homotopy_log.png")
println("saved to '$dir/$(pa.game_name)_λ_list=($λ_list)_homotopy_log.png'")
IJulia.display(p1)

# plot single axis (linear-scaled)
p2 = plot(title="ρ=($ρ), α=($α) \n λ_list=($λ_list)")
for i in eachindex(lambda_vals_list)
    lambda_vals = lambda_vals_list[i]
    plot!(findall(x->x==lambda_vals[1], lambda_vals_list_flattened), lambda_vals, color = lambda_colors[i], marker=:circle, label="λ=$(lambda_vals[1])", leg=:bottomleft)
end        
plot!(ψ_vals_exact_list, c=:red, linestyle=:dash, label="ψ(x)_exact", leg=:bottomleft)
hline!([ϵ], label="ϵ=$ϵ", c=:grey)
savefig("$dir/$(pa.game_name)_λ_list=($λ_list)_homotopy_linear.png")
println("saved to '$dir/$(pa.game_name)_λ_list=($λ_list)_homotopy_linear.png'")
println("--------------------------------\n")
IJulia.display(p2)

In [ ]:
using JLD2

# instantiate a routing game (p, E, s) with desired Nash sol x̂
game_name, g, p, E, E_diag, s_reduced, x̂ = grid_graph3_4_players_reduced()

# assign parameters
α = 0.005
ϵ = 1e-3
ρ = 0.5
max_iter = 10
@show λ_list = [1.0/(2^i) for i in 1:8]

# create an individual folder under homotopy_results/
dir = "homotopy_results/$(game_name)"; mkpath(dir) # mkdir if not exists

In [ ]:
""" RUNNING EXP: PARAMETER CHOICE """
# calling method
println("----------- $(game_name)_ρ=($ρ)_λ_list=($λ_list)_α=($α)_ϵ=($ϵ) -----------")
x, b, C, ψ_vals_list, violation_metrics_list, lambda_vals_list, v, ∇̂ψ_C_norm_list, D_norm_list, J_norm_list, pinv_J_norm_list, F_norm_list = homotopy_exp_parameter_choice(p, E, E_diag, s_reduced, x̂, λ_list, α, ϵ, ρ, max_iter)

# save data to folder
println("saving result to '$dir/$(game_name)_λ_list=($λ_list)_homotopy.jld2'")
@save "$dir/$(game_name)_λ_list=($λ_list)_homotopy.jld2" ρ λ_list α ϵ max_iter lambda_vals_list ψ_vals_list violation_metrics_list v ∇̂ψ_C_norm_list D_norm_list J_norm_list pinv_J_norm_list F_norm_list
println("--------------------------------")

@show findall(x->x>=0.2, x[1:24])   # expected [9, 13]

In [ ]:
using Plots, IJulia
mkpath("current_homotopy")

@load "$dir/$(game_name)_λ_list=($λ_list)_homotopy.jld2" ρ λ_list α ϵ max_iter lambda_vals_list ψ_vals_list violation_metrics_list v ∇̂ψ_C_norm_list D_norm_list J_norm_list pinv_J_norm_list F_norm_list

# plot single axis (log-scaled)
p1 = plot(lambda_vals_list, yscale=:log10, c=:green, label="λ", leg=:bottomleft, xlabel="iter", ylabel="log10 scaled")
plot!(ψ_vals_list, yscale=:log10, c=:blue, linestyle=:dash, label="ψ(x)", leg=:bottomleft)
plot!(violation_metrics_list, yscale=:log10, c=:red, linestyle=:dash, label="violation metric", leg=:bottomleft)
hline!([1e-3], label="1e-3", c=:grey)
plot!(title="ρ=($ρ), α=($α), ϵ=($ϵ)")
println("saving plot to '$dir/$(game_name)_λ_list=($λ_list)_homotopy.png'")
savefig("$dir/$(game_name)_λ_list=($λ_list)_homotopy.png")
IJulia.display(p1)

# plot helper_metrics_plot
# plot!(pinv_J_norm_list, label="pinv_J_norm_list", yscale=:log10)
p2 = plot(F_norm_list, label="F_norm_list", yscale=:log10)
# plot!(∇̂ψ_C_norm_list, label="∇̂ψ_C_norm_list", yscale=:log10, leg=:bottomleft)
# plot!(D_norm_list, label="D_norm_list", yscale=:log10)
# plot!(J_norm_list, label="J_norm_list", yscale=:log10)
hline!([1e-3], label="1e-3", c=:grey)
println("saving plot to '$dir/$(game_name)_λ_list=($λ_list)_helper_metrics.png'")
savefig("$dir/$(game_name)_λ_list=($λ_list)_helper_metrics.png")
IJulia.display(p2)